In [ ]:
!pip install transformers sentence-transformers gradio pandas

import gradio as gr
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Load dataset
incidents = pd.read_json("incidents.json")

# Load FLAN-T5 for response generation
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# Intent-Response Knowledge Base
RESPONSE_TEMPLATES = {
    "phishing": {
        "prompt": "Generate step-by-step instructions for employee who {action} in {attack_vector}. Include: 1. Immediate action 2. Reporting contact",
        "entities": ["action", "attack_vector"]
    },
    "credential_leak": {
        "prompt": "Create urgent response for {data_type} leaked on {affected_site}. List: 1. Password reset 2. 2FA enable 3. Monitoring steps",
        "entities": ["data_type", "affected_site"]
    }
}

def detect_incident(text):
    # Simple similarity matching (replace with actual ML in production)
    for idx, example in incidents.iterrows():
        if example['intent'] in text.lower():
            return example['intent'], example['entities']
    return "unknown", {}

def generate_response(text):
    intent, entities = detect_incident(text)

    if intent == "unknown":
        return "⚠️ Please clarify your security issue. Examples:\n- 'Clicked phishing link'\n- 'Lost work laptop'"

    # Prepare FLAN-T5 prompt
    template = RESPONSE_TEMPLATES[intent]
    prompt = template["prompt"].format(**entities)

    # Generate with FLAN-T5
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=200)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return f"🚨 {intent.replace('_', ' ').title()} Protocol 🚨\n{response}"

# Gradio Interface
with gr.Blocks(title="Security First Responder") as demo:
    gr.Markdown("## 🔒 Enterprise Incident Response Assistant")

    with gr.Row():
        inp = gr.Textbox(label="Describe your incident", placeholder="e.g. 'Clicked link in suspicious email'")
        out = gr.Textbox(label="Recommended Actions", interactive=False)

    examples = gr.Examples(
        examples=[
            "I entered my password on a fake Microsoft login page",
            "My laptop shows a ransomware warning",
            "Left my work phone in a taxi"
        ],
        inputs=inp
    )

    inp.submit(fn=generate_response, inputs=inp, outputs=out)

demo.launch()

In [ ]:
[
  {
    "text": "I clicked a link in a suspicious email pretending to be from IT support",
    "intent": "phishing",
    "entities": {
      "attack_vector": "email",
      "sender": "IT support",
      "action": "clicked link"
    }
  },
  {
    "text": "Entered my corporate password on a fake Microsoft 365 login page",
    "intent": "credential_leak",
    "entities": {
      "affected_site": "Microsoft 365",
      "data_type": "corporate password",
      "severity": "high"
    }
  },
  {
    "text": "My work laptop shows a popup saying files are encrypted with .lockbit extension",
    "intent": "ransomware",
    "entities": {
      "device": "work laptop",
      "indicator": ".lockbit extension",
      "status": "files encrypted"
    }
  },
  {
    "text": "Left my company iPhone in a taxi last night with client data",
    "intent": "lost_device",
    "entities": {
      "device_type": "iPhone",
      "location": "taxi",
      "data_at_risk": "client data"
    }
  },
  {
    "text": "Received alerts for login attempts from Russia on my VPN account",
    "intent": "unauthorized_access",
    "entities": {
      "service": "VPN",
      "location": "Russia",
      "alert_type": "login attempt"
    }
  },
  {
    "text": "Found an unknown USB drive plugged into the reception computer",
    "intent": "suspicious_usb",
    "entities": {
      "device_location": "reception computer",
      "device_status": "plugged in",
      "action_taken": "none"
    }
  },
  {
    "text": "Accidentally emailed client contracts to personal Gmail account",
    "intent": "data_leak",
    "entities": {
      "data_type": "client contracts",
      "leak_vector": "personal Gmail",
      "intent": "accidental"
    }
  },
  {
    "text": "Someone called pretending to be from HR asking for my AD credentials",
    "intent": "social_engineering",
    "entities": {
      "method": "phone call",
      "impersonating": "HR",
      "request": "AD credentials"
    }
  },
  {
    "text": "Noticed unusual network activity when connected to airport WiFi",
    "intent": "wifi_eavesdropping",
    "entities": {
      "location": "airport",
      "indicator": "unusual network activity",
      "data_at_risk": "login credentials"
    }
  },
  {
    "text": "Saw a colleague taking photos of production servers with their phone",
    "intent": "insider_threat",
    "entities": {
      "action": "taking photos",
      "target": "production servers",
      "device": "personal phone"
    }
  }
]